In [1]:
import pandas as pd
import numpy as np
import re

from itertools import combinations
from fuzzywuzzy import fuzz

import ast

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

import difflib
import os

In [2]:
df = pd.read_csv("train.csv")

### Remove wrong chars in dataset using REGEX

In [3]:
characters_to_remove = ['\r', '\n', '\t']

def remove_characters(s):
    for char in characters_to_remove:
        s = s.replace(char, ' ')
    return s

def clean_value(value):
    if isinstance(value, list):
        return [remove_characters(item) if isinstance(item, str) else item for item in value]
    else:
        return remove_characters(value)

def replace_special_sequences(s):
    if isinstance(s, str):
        # Replace 'w\\/' with 'with'
        s = s.replace('w\\/', 'with')
        # Replace '\\/' with '/'
        s = s.replace('\\/', '/')
        # Replace '\\' with '/'
        s = s.replace('\\\\', '\/')
    return s

def remove_u_x(value):
    # Remove \u20 followed by four characters
    value = re.sub(r'\\u\w\w\w\w', '', value)
    # Remove \x0 followed by two characters
    value = re.sub(r'\\x\w\w', '', value)
    return value

def test(x):
    dictionary_extracted = ast.literal_eval(x)
    cleaned_dict = {remove_characters(key): clean_value(value) for key, value in dictionary_extracted.items()}
    return str({replace_special_sequences(key): replace_special_sequences(value) for key, value in cleaned_dict.items()})

df["product_description"] = df["product_description"].map(lambda x: remove_u_x(test(x)))


### Get data and columns from original dataset and make a new one

In [4]:
columns = set()

def make_separate(x):
    dictionary_extracted = ast.literal_eval(x)
    for item in dictionary_extracted.keys():
        columns.add(item)
    return x    


df["product_description"].map(make_separate)
new_df_columns = pd.DataFrame(columns=list(columns))
# new_df_colmns
df = pd.concat([df, new_df_columns], axis=1)

In [5]:
df

,id,product_description,price,IGMP,تعداد فن نصب شده,سازنده مادربرد,کل حافظه گرافیکی,شدت جریان,باس رابط,قابلیتهای تجهیزات ویدیویی,...,نوع دوشاخه برق,جایگاه فن در قسمت پایینی,قدرت موتور,رنگ,ولتاژ,وزن ماوس,ولتاژ اسمی ورودی,توضیحات دوربین,برند,مادربرد
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet /3M PC S...,990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68835,68835,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",2530000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68836,68836,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",22000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68837,68837,"{'دسته بندی': 'شارژر تبلت و موبایل', 'برند': '...",4000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68838,68838,"{'کابل همراه': ['لایتنینگ'], 'ولتاژ خروجی': ['...",4300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def add_data(x):
    dictionary_extracted = ast.literal_eval(x["product_description"])
    for item in dictionary_extracted.keys():
        x[item] = dictionary_extracted[item]
    return x  

df = df.apply(add_data, axis=1)

In [7]:
# #####Remove bad data from price
# min_value_count = (df['price'] == df['price'].min()).sum()
# min_value_count
# under_1000_count = (df['price'] < 1000).sum()
# df[(df['price'] < 5000) & (df['price'] > 0)]

# df[(df['price'] < 10000) & (df['price'] > 0)][['دسته بندی','price']]

### Find Category with few data or can be merged

In [8]:
completely_null_columns = df.columns[df.isnull().all()]

# Display the completely null columns
print("Completely null columns:")
print(completely_null_columns)


Completely null columns:
Index([], dtype='object')


In [9]:
grouped_data = df.groupby('دسته بندی').size().reset_index(name='row_count')
grouped_data.sort_values(by='row_count', ascending=True)

# Filter categories where row_count is under 20
under_20_categories = grouped_data[grouped_data['row_count'] < 20]

# Count the number of distinct categories
num_under_20_categories = under_20_categories.shape[0]

# Display the result
print(f"Number of distinct categories with row_count under 20: {num_under_20_categories}")

Number of distinct categories with row_count under 20: 28


In [10]:
# Filter rows with row_count under 20
category_column = 'دسته بندی'
categories_to_remove = grouped_data[grouped_data['row_count'] < 20][category_column].tolist()
df = df[~df[category_column].isin(categories_to_remove)]

In [11]:
### پیدا کردن دسته بندی هایی که ستون یکسان دارند

category_column = 'دسته بندی'  # Replace with the actual column name

# Create a list to store the common columns for each category
common_columns_by_category = {}

# Iterate over each category
for category in df[category_column].unique():
    # Filter the DataFrame for the current category
    category_df = df[df[category_column] == category]

    # Identify columns with non-null values for the current category
    non_null_columns = category_df.columns[category_df.notnull().any()].tolist()

    # Store the non-null columns in the dictionary
    common_columns_by_category[category] = non_null_columns

# Initialize a dictionary to store duplicate categories and their indices
duplicate_categories = {}

# Loop through each category and its non-null columns
for category1, columns1 in common_columns_by_category.items():
    for category2, columns2 in common_columns_by_category.items():
        if category1 != category2 and set(columns1) == set(columns2):
            # Found two categories with the same non-null columns
            if category1 not in duplicate_categories:
                duplicate_categories[category1] = {"categories": [category2], "columns": columns1}
            else:
                duplicate_categories[category1]["categories"].append(category2)

# Display the results
for category, data in duplicate_categories.items():
    print(f"Category= {category}")
    print(f"Common Columns: {data['columns']}")
    print(f"Duplicate Categories: {data['categories']}")
    print("\n")



Category= هود لنز
Common Columns: ['id', 'product_description', 'price', 'ویژگی ها', 'دسته بندی', 'برند']
Duplicate Categories: ['کابل رابط دوربین', 'شارژر باتری لیتیومی دوربین', 'بند آویز و کمربند دوربین']


Category= کابل رابط دوربین
Common Columns: ['id', 'product_description', 'price', 'ویژگی ها', 'دسته بندی', 'برند']
Duplicate Categories: ['هود لنز', 'شارژر باتری لیتیومی دوربین', 'بند آویز و کمربند دوربین']


Category= شارژر باتری لیتیومی دوربین
Common Columns: ['id', 'product_description', 'price', 'ویژگی ها', 'دسته بندی', 'برند']
Duplicate Categories: ['هود لنز', 'کابل رابط دوربین', 'بند آویز و کمربند دوربین']


Category= بند آویز و کمربند دوربین
Common Columns: ['id', 'product_description', 'price', 'ویژگی ها', 'دسته بندی', 'برند']
Duplicate Categories: ['هود لنز', 'کابل رابط دوربین', 'شارژر باتری لیتیومی دوربین']




In [12]:
### مرج کردن دسته بندی هایی که ستون های مشترک داره

category_column = 'دسته بندی'  # Replace with the actual column name

# Loop through each category and its duplicate categories
for category1, data1 in duplicate_categories.items():
    for category2 in data1["categories"]:
        # Get the indices of rows to update
        indices_to_update = df[df[category_column].isin([category1, category2])].index

        # Create a new concatenated list of categories
        concatenated_categories = sorted(set([category1] + data1['categories']))

        # Set the new concatenated list of categories for the selected rows
        df.loc[indices_to_update, category_column] = ', '.join(concatenated_categories)

In [13]:
#### پیدا کردن دسته هایی که ستون های مشترک زیادی دارن و قیمت ها شبیه است میخواهیم مرج کنیم تا تعداد دسته ها کمتر شود

In [14]:
category_column = 'دسته بندی' 

# Group by category and get the column indices with at least one non-null value
non_null_columns_indices_by_category = df.groupby(category_column).apply(lambda x: np.nonzero(x.notnull().any().values)[0].tolist())

# Calculate the threshold for common indices
common_indices_threshold = 0.8

# List to store pairs of categories with common indices
common_indices_categories = []

# Iterate through each pair of categories and find those with at least 80% common indices
for i, (category1, indices1) in enumerate(non_null_columns_indices_by_category.items()):
    for category2, indices2 in list(non_null_columns_indices_by_category.items())[i+1:]:
        common_indices = set(indices1) & set(indices2)
        common_indices_percentage = len(common_indices) / max(len(indices1), len(indices2))
        
        if common_indices_percentage >= common_indices_threshold:
            common_indices_categories.append((category1, category2, common_indices_percentage))

# # Display the result
# for category1, category2, common_indices_percentage in common_indices_categories:
#     print(f"For Categories {category1} and {category2}, Common Indices Percentage: {common_indices_percentage:.2%}")


In [15]:
# Assuming common_indices_categories is the list containing pairs of categories with common indices
# Each entry is a tuple (category1, category2, common_indices_percentage)

# Initialize a dictionary to store categories and their corresponding common indices
category_indices_dict = {}

# Populate the dictionary
for category1, category2, common_indices_percentage in common_indices_categories:
    if category1 not in category_indices_dict:
        category_indices_dict[category1] = set()
    if category2 not in category_indices_dict:
        category_indices_dict[category2] = set()

    category_indices_dict[category1].add(category2)
    category_indices_dict[category2].add(category1)

# Function to find groups of categories with at least 80% common indices
def find_category_groups(category_indices_dict, common_indices_threshold):
    visited = set()
    groups = []

    def dfs(category, group):
        if category not in visited:
            visited.add(category)
            group.add(category)

            for neighbor in category_indices_dict[category]:
                if neighbor not in visited:
                    dfs(neighbor, group)

    for category in category_indices_dict:
        if category not in visited:
            group = set()
            dfs(category, group)
            groups.append(group)

    return groups

# Find groups of categories with at least 80% common indices
common_indices_threshold = 0.8
category_groups = find_category_groups(category_indices_dict, common_indices_threshold)

# Display the result
for i, group in enumerate(category_groups, start=1):
    print(f"Category Group {i}: {group}")


Category Group 1: {'آداپتور و کانورتر', 'محافظ صفحه نمایش تبلت'}
Category Group 2: {'لوازم جانبی ساعت و مچ بند هوشمند', 'لوازم جانبی دوربین های امنیتی و نظارتی', 'سر سه پایه', 'قطعات جانبی موبایل و تبلت', 'قطعات یدکی موبایل و تبلت', 'سافت باکس', 'لوازم جانبی عکاسی و فیلمبرداری', 'کیف دوربین', 'سایر لوازم جانبی لپ تاپ', 'لوازم جانبی پروژکتور', 'لوازم جانبی تجهیزات ذخیره سازی', 'قلم لمسی (Stylus)', 'لوازم جانبی قطعات کامپیوتر', 'لوازم جانبی مک بوک', 'کیف و کاور لپ تاپ', 'سایر لوازم جانبی دوربین', 'ابزار شبکه', 'رفلکتور'}
Category Group 3: {'لوازم جانبی پرینتر لیبل زن و حرارتی', 'استند تبلت', 'بند آویز و کمربند دوربین, شارژر باتری لیتیومی دوربین, هود لنز, کابل رابط دوربین', 'سایر لوازم جانبی تلفن', 'کیت تمیز کننده'}
Category Group 4: {'تونر', 'کارتریج و جوهر مخزن', 'سایر لوازم جانبی پرینتر'}
Category Group 5: {'هدست تلفن', 'دسته بازی'}
Category Group 6: {'کابل صوتی و تصویری', 'کابل افزایش طول'}


In [16]:
# Find price similarity in each category
category_column = 'دسته بندی'  # Replace with the actual column name
price_column = 'price'  # Replace with the actual column name

# Create a dictionary to store the mean price for each category
category_mean_prices = {}

# Populate the dictionary with mean prices for each category
for category, group in category_indices_dict.items():
    category_mean_prices[category] = df[df[category_column].isin(group)][price_column].mean()

# Display the mean prices for each category within each group
for i, group in enumerate(category_groups, start=1):
    print(f"Category Group {i}:")
    for category in group:
        mean_price = category_mean_prices.get(category, "N/A")
        print(f"  Category: {category}, Mean Price: {mean_price}")


Category Group 1:
  Category: آداپتور و کانورتر, Mean Price: 1275750.850086655
  Category: محافظ صفحه نمایش تبلت, Mean Price: 2562332.2580645164
Category Group 2:
  Category: لوازم جانبی ساعت و مچ بند هوشمند, Mean Price: 1478301.5612552806
  Category: لوازم جانبی دوربین های امنیتی و نظارتی, Mean Price: 3112737.81106383
  Category: سر سه پایه, Mean Price: 4744690.844840764
  Category: قطعات جانبی موبایل و تبلت, Mean Price: 3752516.6915682503
  Category: قطعات یدکی موبایل و تبلت, Mean Price: 9740543.190677967
  Category: سافت باکس, Mean Price: 11376601.295165394
  Category: لوازم جانبی عکاسی و فیلمبرداری, Mean Price: 3644301.716724537
  Category: کیف دوربین, Mean Price: 7787458.846994535
  Category: سایر لوازم جانبی لپ تاپ, Mean Price: 8038317.404688464
  Category: لوازم جانبی پروژکتور, Mean Price: 2752922.944598338
  Category: لوازم جانبی تجهیزات ذخیره سازی, Mean Price: 4147039.2330129268
  Category: قلم لمسی (Stylus), Mean Price: 3439680.76911315
  Category: لوازم جانبی قطعات کامپیوتر,

In [17]:
category_column = 'دسته بندی'  

# Iterate through each group and update the values in the 'دسته بندی' column
for i, group in enumerate(category_groups, start=1):
    concatenated_categories = "_".join(sorted(df[df[category_column].isin(group)][category_column].unique()))
    df.loc[df[category_column].isin(group), category_column] = concatenated_categories

In [18]:
#Drop full null

In [19]:
# Assuming new_df is your DataFrame
completely_null_columns = df.columns[df.isnull().all()]

# Display the completely null columns
print("Completely null columns:")
print(completely_null_columns)


Completely null columns:
Index(['سرویسهای IP', 'حساسیت برد', 'کانکتور Mini SAS', 'تعدادپورت GSM/3G',
       'پورت شبکه', 'توضیحات امکانات', 'پروتکل مدیریت',
       'توضیحات چاپ دوروی خودکار', 'وضوح چاپ', 'پورت USB',
       ...
       'درجه چرخش', 'ابعاد برد', 'سازگار با دستگاههای', 'زمان شارژ خازن',
       'تعداد VoIP Trunks', 'پورت مدیریت تحت شبکه RJ45', 'کلاس منبع تغذیه',
       'تعداد نمایشگر قابل نصب', 'مشخصات مواد مصرفی', 'جنس صندلی'],
      dtype='object', length=153)


In [20]:
df = df.drop(columns=completely_null_columns)

#### Delete columns with less than 5 data

In [21]:
# Iterate through each column and count non-null values
columns_under_5_samples = []
for column in df.columns:
    non_null_count = df[column].count()
    if non_null_count < 5:
        columns_under_5_samples.append(column)
print(len(columns_under_5_samples))

73


In [22]:
df = df.drop(columns=columns_under_5_samples)

In [23]:
# df=df_filtered_dropped.copy()

### Find Similar Category name to merge

In [24]:
# Set a similarity threshold for identifying similar column names
similarity_threshold = 90

# Get all combinations of column names
column_combinations = list(combinations(df.columns, 2))

# Find similar column name pairs
similar_columns = [(col1, col2) for col1, col2 in column_combinations if fuzz.ratio(col1, col2) > similarity_threshold]

In [25]:
# # Print similar column name pairs
# print("Similar Column Name Pairs:")
# for col1, col2 in similar_columns:
#     print(f"column1='{col1}'")
#     print(f"column2='{col2}'\n")

In [26]:
def group_similar_columns(similar_columns, df):
    grouped_columns = []

    for col1, col2 in similar_columns:
        # Find rows where at least one of the two columns has values
        mask = df[col1].notnull() & df[col2].notnull()

        # Check if there are at most 5 rows where both columns have values
        if 0 <= sum(mask) <= 3:
            # Check if the columns are already in a group
            col1_group = [group for group in grouped_columns if col1 in group]
            col2_group = [group for group in grouped_columns if col2 in group]

            # If neither column is in a group, create a new group
            if not col1_group and not col2_group:
                grouped_columns.append([col1, col2])
            # If one column is already in a group, add the other column to that group
            elif col1_group and not col2_group:
                col1_group[0].append(col2)
            elif col2_group and not col1_group:
                col2_group[0].append(col1)
            elif col1_group != col2_group:
                # If both columns are in different groups, merge the groups
                grouped_columns.remove(col1_group[0])
                grouped_columns.remove(col2_group[0])
                grouped_columns.append(col1_group[0] + col2_group[0])

        elif 3 < sum(mask) <= 5:
            # Check if 'دسته بندی' values are the same for both columns
            category_mask = df.loc[mask, 'دسته بندی'].nunique() == 1

            if category_mask:
                # Check if the columns are already in a group
                col1_group = [group for group in grouped_columns if col1 in group]
                col2_group = [group for group in grouped_columns if col2 in group]

                # If neither column is in a group, create a new group
                if not col1_group and not col2_group:
                    grouped_columns.append([col1, col2])
                # If one column is already in a group, add the other column to that group
                elif col1_group and not col2_group:
                    col1_group[0].append(col2)
                elif col2_group and not col1_group:
                    col2_group[0].append(col1)
                elif col1_group != col2_group:
                    # If both columns are in different groups, merge the groups
                    grouped_columns.remove(col1_group[0])
                    grouped_columns.remove(col2_group[0])
                    grouped_columns.append(col1_group[0] + col2_group[0])

    return grouped_columns

grouped_columns = group_similar_columns(similar_columns, df)

In [27]:
# # Print the resulting groups
# for group in grouped_columns:
#     print(f"Group: {group}")
#     print("\n")

In [28]:
def find_common_subsequence(strings):
    # Find common prefix among a list of strings
    common_prefix = os.path.commonprefix(strings)

    # Find common suffix among a list of strings (reversed)
    reversed_strings = [s[::-1] for s in strings]
    common_suffix_reversed = os.path.commonprefix(reversed_strings)[::-1]

    # The common subsequence is the combination of common prefix and common suffix
    common_subsequence = common_prefix + common_suffix_reversed

    if common_subsequence:
        return common_subsequence
    else:
        return None


def merge_columns_within_groups(df, grouped_columns):
    merged_values = {}

    for group in grouped_columns:
        print(group)
        # Find rows where at least one column in the group has values
        mask = df[group].notnull().any(axis=1)

        # Use the common substring of the column names as the new column name
        common_substring = find_common_subsequence(group)
        new_column_name = f"{common_substring}_Merged" if common_substring else 'Merged'
        print(new_column_name)

        # Concatenate values with ' | ' separator, excluding NaN values using apply
        df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)

        # Store merged values for all columns in the group
        # merged_values[new_column_name] = df.loc[mask, new_column_name].tolist()

        # Drop the original columns
        df.drop(columns=group, inplace=True, errors='ignore')  # Ignore errors if columns don't exist

    # Concatenate all columns at once to improve performance
    df = pd.concat([df, df[new_column_name]], axis=1)
    df.drop(columns=[new_column_name], inplace=True, errors='ignore')

    return df#, merged_values

# df, merged_values = merge_columns_within_groups(df, grouped_columns)
df = merge_columns_within_groups(df, grouped_columns)


['کارتهای حافظهی قابل پشتیبانی', 'کارت حافظه قابل پشتیبانی']
کارت قابل پشتیبانی_Merged


/tmp/ipykernel_15062/752076284.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)


['نوع صفحه نمایش', 'نوع صفحهنمایش']
نوع صفحهنمایش_Merged


/tmp/ipykernel_15062/752076284.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)


['قابلیتهای کیس', 'قابلیتهای کپی']
قابلیتهای ک_Merged


...

/tmp/ipykernel_15062/752076284.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)


['نسخهی بلوتوث', 'نسخه بلوتوث']
نسخه بلوتوث_Merged


/tmp/ipykernel_15062/752076284.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)


['سایرقابلیتها', 'سایر قابلیتها']
سایرقابلیتها_Merged


/tmp/ipykernel_15062/752076284.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df.loc[mask, group].astype(str).apply(lambda row: ' | '.join(cell for cell in row if cell != 'nan'), axis=1)


### Work on each column to find wrong data(s:1-using bert, 2-using clustering)

#### Bert(but the solution was not chosen)

In [29]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens")
# model = AutoModel.from_pretrained("m3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens")

In [30]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [31]:
# import pandas as pd
# from sentence_transformers import SentenceTransformer
# import numpy as np

# def calculate_semantic_similarity(dataframe, column_name):
#     # Load a pre-trained Farsi language model
#     model = SentenceTransformer('m3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens')
#     # model = SentenceTransformer('HooshvareLab/bert-base-parsbert-uncased')

#     # Encode the column name
#     column_name_embedding = model.encode(column_name)

#     # Encode each value in the specified column
#     dataframe['embedding'] = dataframe[column_name].apply(lambda x: model.encode(x))

#     # Calculate cosine similarity between each value and the column name
#     dataframe['similarity'] = dataframe['embedding'].apply(lambda x: np.dot(column_name_embedding, x) / (np.linalg.norm(column_name_embedding) * np.linalg.norm(x)))

#     return dataframe[[column_name, 'similarity']]


# result_df = calculate_semantic_similarity(test_df, 'وزن')


In [32]:
# result_df[(result_df['similarity'] > 0.55) & (result_df['similarity'] < 0.6)]

#### Using Kmeans and cluster the vlues to find true data

In [ ]:
###Nans cluster manually

def cluster_column(df, column, k=3):

    # Check if the column contains lists
    df[column] = df[column].apply(lambda x: tuple(x) if isinstance(x, list) else x)

        
    # Convert the column to string to handle non-string entries
    string_data = df[column].astype(str)
    
    # Identify entries that contain at least one numeric character
    mixed_mask = string_data.str.contains('\d')
    
    # Convert mixed entries to numeric values
    numeric_values = pd.to_numeric(df.loc[mixed_mask, column], errors='coerce')
    
    # Use LabelEncoder to convert strings to numerical labels for non-numeric entries
    label_encoder = LabelEncoder()
    encoded_data = label_encoder.fit_transform(string_data[~mixed_mask])
    
    # Reshape the data for K-means
    data = encoded_data.reshape(-1, 1)
    
    # Initialize the KMeans model
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    
    # Fit the model to the non-numeric data
    kmeans.fit(data)
    
    # Add the cluster labels to the DataFrame for non-numeric entries
    df[column + '_cluster_labels'] = -1  # Initialize with -1
    df.loc[~mixed_mask, column + '_cluster_labels'] = kmeans.labels_

for column in df.columns[3:]:
    print(f"Column Number: {df.columns.get_loc(column)}")
    cluster_column(df, column)
    
    # # Check if cluster -1 has values for the current column
    cluster_label_column = column + '_cluster_labels'
    # if cluster_label_column in df.columns:
    cluster_minus_one_values = df[df[cluster_label_column] == -1][column]
    if not cluster_minus_one_values.empty:
        print("----- There are Unique values in cluster -1: -----")
        num_clusters = len(df[cluster_label_column].unique()) - 1
        
        print(f"Column '{column}' values in cluster -1:")
        if isinstance(cluster_minus_one_values.iloc[0], list):
            # Convert lists to tuples for uniqueness
            unique_values_cluster_minus_one = cluster_minus_one_values.apply(tuple).unique()
            print(unique_values_cluster_minus_one)
        else:
            print(cluster_minus_one_values)
    else:
        num_clusters = len(df[cluster_label_column].unique())
    
    # Print values for each cluster
    for cluster_label in range(num_clusters):
        cluster_values = df[df[cluster_label_column] == cluster_label][column]
        if not cluster_values.empty:
            print(f"Column '{column}' values in cluster {cluster_label}:")
            print(cluster_values)
            print()
            
    # df.drop([cluster_label_column], axis=1, inplace=True)
    print("\n--------------------------------------")

### Save csv file

In [34]:
df.to_csv('data_step1.csv', index=False)